In [48]:
import requests 

from openai import OpenAI
from dotenv import load_dotenv

from elasticsearch import Elasticsearch
from tqdm.auto import tqdm


In [49]:
load_dotenv('../.env')

True

# 

# 2

In [50]:

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [59]:
es_client = Elasticsearch("http://localhost:9200")

es_client.info()

ObjectApiResponse({'name': '35373feb37fe', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'T3O9qZ3xTeCZMuDewssaAQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [60]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}



index_name = "course-questions"


es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [61]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:58<00:00, 16.19it/s]


# 

# 3

In [63]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }


    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit)

    return result_docs




In [64]:
query = "How do I execute a command in a running docker container?"

response = elastic_search(query)

In [65]:
response

[{'_index': 'course-questions',
  '_id': '2GgaR5ABj3SmiRjVZ6Vt',
  '_score': 75.54128,
  '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|--------+------------------+-------+-------|\n| public | yellow_taxi_data | table | root  |\n+--------+------------------+-------+-------+\nSELECT 1\nTime: 0.009s\nroot@pg-database:ny_taxi>',
   '

In [68]:
top_score = response[0]['_score']

print(f'The top score is {top_score}')

The top score is 75.54128


# 

# 4

In [70]:
def elastic_search(query):

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }


    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs


In [71]:
query = "How do I execute a command in a running docker container?"

response = elastic_search(query)

In [73]:
response

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [76]:
third_answer = response[2]['question']

print(f'The third andwer given by the Elastic Search is : \'{third_answer}\'')

The third andwer given by the Elastic Search is : 'How do I copy files from a different folder into docker container’s working directory?'


# 

# 5

In [77]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"""Q: {doc['question']}\nA: {doc['text']}\n\n""".strip()
        #"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [80]:
builded_prompt = build_prompt(query, response)

print(builded_prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/p

In [81]:
print(f"The length of the builded prompt is "+str(len(builded_prompt)))

The length of the builded prompt is 1458


# 

# 6

In [82]:
import tiktoken

In [83]:
encoding = tiktoken.encoding_for_model("gpt-4o")


In [84]:
encoded = encoding.encode(builded_prompt)

In [87]:
print(f"The tokens of the builded query is "+str(len(encoded)))

The tokens of the builded query is 321


# 

# Optional 1

In [89]:
client = OpenAI()

In [90]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [91]:
answer = llm(builded_prompt)

In [92]:
print(answer)

To execute a command in a running docker container, you can first find the container id using the command `docker ps` and then use the command `docker exec -it <container-id> bash` to run the desired command in that specific container.


# 

# Optional 2

In [100]:
avg_tokens_input = 150
avg_tokens_output = 250

total_requests = 1000

input_tokens_price = 0.005
output_tokens_price = 0.015


total_input_tokens = avg_tokens_input * total_requests
total_output_tokens = avg_tokens_output * total_requests

total_input_tokens_price = (total_input_tokens / 1000) * input_tokens_price
total_output_tokens_price = (total_output_tokens / 1000) * output_tokens_price


total_price = total_input_tokens_price + total_output_tokens_price

In [101]:
print(f"The total price is ${total_price:.2f}")

The total price is $4.50
